## Example showing the use of Xarray to read OpenDAP Netcdf files from Sentinel

In [ ]:
%matplotlib widget

import numpy as np
import xarray as xa
import matplotlib.pyplot as plt
from skimage import exposure # We are going to rescale the image

We have now imported some libraries and will load some Sentinel 2 OpenDAP NetCDF
data from https://satellittdata.no

In [ ]:
# PODAAC
# url = "https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2021/308/20211104090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"

# url = "https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L2P/AVHRRMTA_G/NAVO/v2/2021/261/20210918005147-NAVO-L2P_GHRSST-SST1m-AVHRRMTA_G-v02.0-fv02.0.nc"

# Fra satellittdata.no
url = 'https://nbstds.met.no/thredds/dodsC/DC-S2-32VMK-2020_BB'
dset = xa.open_dataset(url+'#fillmismatch') # opening OpenDAP NetCDF dataset


Having opened the dataset we can how have a look at the header

In [ ]:
display(dset)

In [ ]:
display(dset.B2)

**What is B2?**

We can plot B2 to see it better

In [ ]:
# East - west
xmin = 9000
xmax = -1

# North - south
ymin = 0
ymax = 2000

plt.figure()
view =dset.B2[0,ymin:ymax,xmin:xmax]
view.plot(cmap='Greys',vmax=1000)
plt.show()



In [ ]:
print(dset.TCI)

In [ ]:

extent = np.asarray(
            [dset.lon[ymin,xmin].data, dset.lon[ymin,xmax].data,
            dset.lat[ymax,xmax].data, dset.lat[ymin,xmin].data]
         )

#print('lon and lat',(extent)) # These are the latitudes and longitudes
n=0
# Reading in some parts of the r b g bands
b = dset.B2[n,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])
g = dset.B3[n,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])
r = dset.B4[n,ymin:ymax,xmin:xmax]/float(dset.attrs['QUANTIFICATION_VALUE'])

P0, P1 = (0, 0.15) # These are the percentages we are going to rescale on 

# Rescaling the exposures using scikit-image 
r = exposure.rescale_intensity(r,in_range=(P0,P1))
g = exposure.rescale_intensity(g,in_range=(P0,P1))
b = exposure.rescale_intensity(b,in_range=(P0,P1))

# Making the RGB image
rgb = np.dstack((r,g,b))

# Clipping it
rgb[rgb>1]=1.0


# Plotting the RGB image
plt.figure()
plt.imshow(rgb, interpolation='none', aspect='auto', extent = extent)
plt.show()